In [ ]:
"""
network.py
~~~~~~~~~~

A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random
random.seed(250)

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.rand(y, 1) for y in sizes[1:]]
        self.weights = [np.random.rand(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = self.sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,                    # SGD is an gradient descent optimization function, which controls the descent steps.
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):                                                   # this loop is for gradient descent
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.w_update_batch(mini_batch, eta)
            if test_data:
                print ("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
            else:
                print ("Epoch {0} complete".format(j))

    def w_update_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:                                                   # this loop is to calcuate the gradients of the whole bactch in on gradient descent step
            delta_nabla_b, delta_nabla_w = self.backpropagation(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]         # accumulate gradients
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-eta*nw/len(mini_batch)                                  # take aveage gradients of the mini_batch
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-eta*nb/len(mini_batch)
                       for b, nb in zip(self.biases, nabla_b)]

    def backpropagation(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b                               # z is the current node middle state (input to activation) , neither the prior node output, nor the current node output
            zs.append(z)                                              # zs contain the middle state of all hidden layers, and the output layer
            activation = self.sigmoid(z)                              # activation is the output of each layer
            activations.append(activation)                            # activations contain outputs of all layers, including input and output layer
        # backward pass                                               # delta is theta, which is the loss function derivative against current node activation input Z
        delta = self.cost_derivative(activations[-1], y)*self.sigmoid_prime(zs[-1])    # Output layer: delta = formula (2.8), which is derived from fromula (BP1); delta is the delta out of current layer

        nabla_b[-1] = delta                                           # Output layer: derivative b = formula (BP3)
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())      # Output layer: derivative w = formula (2.10), which is derived from fromula (BP4)
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            delta = np.dot(self.weights[-l+1].transpose(), delta)*self.sigmoid_prime(z)   # Hidden layer: delta = formula (BP2)

            nabla_b[-l] = delta                                         # Hidden layer: derivative b = formula (BP3)
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())  # Hidden layer: derivative w = formula (2.10), which is derived from fromula (BP4)
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

      #### Miscellaneous functions
    def sigmoid(self, z):
        """The sigmoid function."""
        return 1.0/(1.0+np.exp(-z))

    def sigmoid_prime(self, z):
        """Derivative of the sigmoid function."""
        return self.sigmoid(z)*(1-self.sigmoid(z))

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('/content/gdrive/My Drive/breast_cancer/breast-cancer.csv')

def normalization(vec):
  return (vec-min(vec)) / (max(vec) - min(vec))

x_data = data.drop(['id', 'diagnosis'], axis=1)
x_data = np.array(x_data.apply(normalization))

train_test_split = int(len(data) * 0.75)
x_train = x_data[:train_test_split]
x_test = x_data[train_test_split:]

mapping = {'M':0, 'B':1}
y_data = np.array(data['diagnosis'].replace(mapping))
y_train = y_data[:train_test_split]
y_test = y_data[train_test_split:]

In [ ]:
nn_structure = [x_train.shape[1],
                x_train.shape[1],
                int(x_train.shape[1]/2),
                1]
nn = Network(nn_structure)
learning_rate = 0.001
training_data = list(zip(x_train, y_train))
test_data = list(zip(x_test, y_test))
nn.SGD(training_data, 10, 50, learning_rate, test_data)

Epoch 0: 1 / 143
Epoch 1: 1 / 143
Epoch 2: 1 / 143
Epoch 3: 1 / 143
Epoch 4: 1 / 143
Epoch 5: 1 / 143
Epoch 6: 1 / 143
Epoch 7: 1 / 143
Epoch 8: 1 / 143
Epoch 9: 1 / 143


In [ ]:
sizes = [3,3,1]
[np.random.rand(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

[array([[0.24390526, 0.62290945, 0.88543243],
        [0.13841514, 0.23140227, 0.02043831],
        [0.24376074, 0.19075641, 0.0307523 ]]),
 array([[0.99655741, 0.60242146, 0.28976407]])]